In [1]:
algos = ['LightGBM', 'Xgboost', 'CatBoost', 'Neural Network']

from supervised.automl import AutoML

from helpers import *

training_filters = [
    {
        'feature': 'month',
        'operator': 'between',
        'value': [3, 9]
    }
]

validation_filters = [
    {
        'feature': 'date_forecast',
        'operator': 'between',
        'value': ['2020-05-01', '2020-07-01']
    }
]

test_filters = [
    {
        'feature': 'date_forecast',
        'operator': 'between',
        'value': ['2021-05-01', '2021-07-01']
    }
]


agg_strats = ['min','max','mean','std']

m_wrap = DataWrapper(
    impute_strategy = 'fbfill',
    aggregation_strats = agg_strats,
    y_scalers = [Y_Scaler_MaxAbs_per_building()],
    transform_pred_strategy = [],
    training_filters=training_filters,
    validation_filters=validation_filters,
    test_filters=test_filters,
    bagging_filters=False, # these are applied additionally to the other filters on val, test, sub
    )

X_train_agg, y_train_agg = m_wrap.get_train(aggregated=True,drop_val=False,drop_test=False,y_scaled=True)
X_sub_agg = m_wrap.get_sub(aggregated=True)

train_idx = X_train_agg[~X_train_agg['date_forecast'].between('2020-05-01', '2020-07-01')].index
val_idx = X_train_agg[X_train_agg['date_forecast'].between('2020-05-01', '2020-07-01')].index

X_train_agg = X_train_agg.select_dtypes(exclude=['datetime','timedelta','period[M]'])

cv = [(train_idx, val_idx)]

mod = AutoML(
    results_path='AutoML_Results_TEST/',
    mode='Compete',
    total_time_limit=60*60*8,
    eval_metric='mae',
    algorithms=algos,
    train_ensemble=True,
    stack_models=True,
    verbose=2,
    random_state=42,
    validation_strategy={"validation_type": "custom"}
)

mod.fit(X_train_agg, y_train_agg, cv=cv)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


AutoML directory: AutoML_Results_TEST/
The task is regression with evaluation metric mae
AutoML will use algorithms: ['LightGBM', 'Xgboost', 'CatBoost', 'Neural Network']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree: trained.
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models


KeyboardInterrupt: 